#**Library Insatallation**

In [1]:
!pip install -U unsloth unsloth-zoo transformers peft accelerate --quiet
!pip install trl==0.23.0 --quiet
!pip install fastapi uvicorn pyngrok nest-asyncio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#**Authentication**

In [2]:
from huggingface_hub import login
hf_token = ""
ngrok_token = ""
login(hf_token)

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from unsloth import FastLanguageModel
import torch
from huggingface_hub import login
import nest_asyncio
from pyngrok import ngrok
import nest_asyncio
import os
import re
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import torch

/tmp/ipython-input-1032134006.py:4: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


#**Model Loading**

In [4]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

In [11]:
path = "Fatma5000/llama_arabic_medical"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=path,
    max_seq_length=1024,
    token=hf_token,
    quantization_config=bnb_config,
    dtype = None,
    load_in_4bit = True,
)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0


#**Inference Setup**

In [12]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

#**API & Cleaning Setup**
**Initialize App**: Sets up the FastAPI application to handle web requests.
CORS Configuration: Enables Cross-Origin Resource Sharing (CORS) to allow your React frontend (running in the browser) to communicate with this backend server without security errors.

**Text Cleaning (clean_medical_text)**: Defines a post-processing function that acts as a filter. It programmatically removes specific unwanted phrases (e.g., "IslamWeb," "Fatwa numbers," "External links") from the model's output before sending it to the user.

**System Prompt**: Defines the core instructions for the AI model (e.g., "You are Dr. Shifaa," "Do not mention sources," "Be concise," "Suggest alternatives for allergies").
#**The Endpoint (/ask)**
**API Endpoint**: Defines the specific route (doorway) that your React app calls via HTTP POST requests.

**Input Handling**: Receives the user's question combined with the chat history (context).

**Inference**: Sends this input to the loaded LLaMA model to generate a response.

**Sanitization**: Passes the raw generated text through the clean_medical_text filter.

**Response**: Returns the final, clean, and formatted answer back to the frontend.
#**Run Server**

In [ ]:
nest_asyncio.apply()

app = FastAPI(title="Shifaa Medical Assistant API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class MedicalQuery(BaseModel):
    question: str
    max_length: int = 256
    temperature: float = 0.5
    top_p: float = 0.9

class MedicalResponse(BaseModel):
    answer: str
    status: str

def clean_medical_text(text: str) -> str:
    forbidden_phrases = [
        "يسرنا تواصلك مع موقع الشبكة الإسلامية",
        "الشبكة الإسلامية",
        "إجابة المستشار",
        "انتهت إجابة",
        "رابط",
        "islamweb",
        "fatwa",
        "وفقك الله وسدد خطاك",
        "والله الموفق",
        "________________",
        "راجع قسم الأسئلة القانونية"
    ]

    # حذف الجمل المحددة
    for phrase in forbidden_phrases:
        text = text.replace(phrase, "")

    text = re.sub(r'رقم\s*\(\d+\s*و?رقم?\d*\)', '', text)
    text = re.sub(r'\(\d+\)', '', text)

    text = re.sub(r'http\S+', '', text)

    text = " ".join(text.split())

    return text

SYSTEM_PROMPT = """<|start_header_id|>system<|end_header_id|>
أنت الدكتور "شِفاء"، مساعد طبي ذكي.
⚠️ تعليمات التنسيق (صارمة جداً):
1. ابدأ الإجابة الطبية مباشرة (لا تقل "يسرنا تواصلك" أو "أهلاً بك في الموقع").
2. تحدث بصيغة المتكلم (أنا) وليس بصيغة المؤسسة (نحن).
3. **ممنوع منعاً باتاً** ذكر كلمات مثل: "الشبكة الإسلامية"، "أرقام استشارات"، "روابط فتاوى".
4. إذا ذكر المريض حساسية، اقترح البديل فوراً.
5. اجعل إجابتك مركزة وقصيرة.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
سياق المحادثة:
{}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

@app.post("/ask", response_model=MedicalResponse)
async def ask_medical_question(query: MedicalQuery):
    try:
        if 'model' not in globals() or 'tokenizer' not in globals():
             raise HTTPException(status_code=503, detail="Model not loaded")

        formatted_prompt = SYSTEM_PROMPT.format(query.question)
        inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=query.max_length,
            temperature=query.temperature,
            top_p=query.top_p,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

        if "<|start_header_id|>assistant<|end_header_id|>" in decoded_output:
            raw_answer = decoded_output.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
            raw_answer = raw_answer.replace("<|eot_id|>", "").strip()

            clean_answer = clean_medical_text(raw_answer)
        else:
            clean_answer = clean_medical_text(decoded_output)

        return MedicalResponse(answer=clean_answer, status="success")

    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

async def run_production_server(port=8000):
    ngrok.kill()
    os.system("killall ngrok")
    if 'ngrok_token' in globals(): ngrok.set_auth_token(ngrok_token)

    try:
        public_url = ngrok.connect(port)
        print(f"\n{'='*50}")
        print(f" {public_url.public_url}/ask")
        print(f"{'='*50}\n")

        config = uvicorn.Config(app=app, host="0.0.0.0", port=port, log_level="error")
        server = uvicorn.Server(config)
        await server.serve()

    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    await run_production_server()